In [1]:
"""тренируем автоэнкодер который работает на внутреннем представлениии кодера для зуба ( 256* 32), 
   метки выдает тот же кодер из T2.
"""
import torch

from points_dataset import EmbedderDataset
from alignment_ae_dataset import AlignerDataset
from torch_geometric.data import Data
from torch.utils.data import DataLoader

from models import *
from models_align import *
from point_cloud_dataset import PointCloudDataset
from torch.utils.tensorboard import SummaryWriter
torch.__version__, "cuda available" if torch.cuda.is_available() else "cpu only"

('2.4.0+cu124', 'cuda available')

In [9]:
device = "cpu" if not torch.cuda.is_available() else "cuda"

model_dir = "models_align"
align_ds_path = r"datasets_align\dataset_256.pth"
align_ds = torch.load(align_ds_path)
# align_ds_data = align_ds.data # нет у конкатенрованного ds такого поля 
print(f"Using {device} device, ds length - {len(align_ds)}")

EPOCHS = 560
BATCH_SIZE = 64
SPLIT_FACTOR = .8
# TRAIN_MODE = (False, True)[1]
# REMOVE_OLD_MODELS = True
N_TEETH = 32
POINTS_PER_SHAPE = 128
POINT_DIM = 3
EMBEDDED_POINT_DIM = 32

hidden_dim = POINTS_PER_SHAPE * 3 * N_TEETH // 6

train_loader = DataLoader(align_ds, batch_size=BATCH_SIZE, shuffle=True)
model = autoencoder(POINTS_PER_SHAPE*3*N_TEETH, hidden_dim)


C:\Users\5010858\AppData\Local\Temp/ipykernel_30484/2105116927.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  align_ds = torch.load(align_ds_path)


Using cuda device, ds length - 10


In [11]:
def train_autoencoder(model, train_loader, device, epochs=EPOCHS, lr=1e-3, log_dir="runs/autoencoder"):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=log_dir)

    best_loss = float('inf')  # Initialize best loss to infinity
    best_model_path = None

    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch_idx, (x,y) in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            x = x.view(-1, POINTS_PER_SHAPE*EMBEDDED_POINT_DIM*N_TEETH ).to(device)
            x

best_model_path = train_autoencoder(model, train_loader, device)
print(f"Best model saved at: {best_model_path}")


Best model saved at: None


In [6]:
256*32*32, 15*32

(262144, 480)